In [5]:
import pynq
from pynq import Overlay
import numpy as np
from pynq import allocate

overlay = Overlay("sumAll.bit")
dma = overlay.axi_dma_0

t = [1, 2 ,3, 4, 5, 6, 7]

array_t = np.array(t)
sample = array_t.astype(np.int32)

in_buffer = allocate(shape=(n,), dtype=np.int32)
out_buffer = allocate(shape=(1,), dtype=np.int32)
np.copyto(in_buffer,sample)

dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)

dma.sendchannel.wait()
dma.recvchannel.wait()

print("Fpga result",out_buffer)

in_buffer.close()
out_buffer.close()


Fpga result [28]


In [1]:
import pynq
from pynq import Overlay
from pynq.lib.dma import DMA
import numpy as np
from pynq import allocate
import time

overlay = Overlay("sumAll.bit")

dma = overlay.axi_dma_0

import numpy as np

random_numbers = np.random.randint(1, 10, size=10000000)

t = random_numbers

n = len(t)
print(n)
sum = 0
sw_exe_time = 0
hw_exe_time = 0
start_time = time.time()
for i in range(n):
    sum = sum + t[i];

stop_time = time.time()
sw_exe_time =  sw_exe_time + stop_time - start_time
print("cpu result", sum)
array_t = np.array(t)
sample = array_t.astype(np.int32)

# Allocate buffers for the input and output signals
with allocate(shape=(n,), dtype=np.int32) as in_buffer, \
     allocate(shape=(1,), dtype=np.int32) as out_buffer:
    np.copyto(in_buffer,sample)
    start_time = time.time()
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    stop_time = time.time()
    hw_exe_time = hw_exe_time + stop_time - start_time
#     print(in_buffer)
    print("Fpga result",out_buffer)
    in_buffer.close()
    out_buffer.close()
    
print("CPU(sum 10000000 numbers) ", sw_exe_time)
print("FPGA(sum 10000000 numbers) ", hw_exe_time)
print("FPGA faster ", sw_exe_time/hw_exe_time)

print(sum)

10000000
cpu result 50001505
Fpga result [50001505]
CPU(sum 10000000 numbers)  42.03763222694397
FPGA(sum 10000000 numbers)  0.10103869438171387
FPGA faster  416.0547845969791
50001505


In [ ]:
import pynq
from pynq import Overlay
from pynq.lib.dma import DMA
import numpy as np
from pynq import allocate
import time

# Load the FPGA overlay
overlay = Overlay("sumAll.bit")
dma = overlay.axi_dma_0

# Generate random numbers between 1 and 10, size=100,000,000
total_elements = 100_000_000
random_numbers = np.random.randint(1, 10, size=total_elements, dtype=np.int32)

# Compute the sum on the CPU for verification
print(f"Total number of elements: {len(random_numbers)}")
cpu_start_time = time.time()
cpu_sum = np.sum(random_numbers)
cpu_end_time = time.time()
cpu_exe_time = cpu_end_time - cpu_start_time
print("CPU sum result:", cpu_sum)
print(f"CPU execution time: {cpu_exe_time:.4f} seconds\n")

# Prepare data for FPGA processing
array_t = np.array(random_numbers, dtype=np.int32)
chunk_size = 1_000_000  # Number of elements per chunk
num_chunks = total_elements // chunk_size
remaining_elements = total_elements % chunk_size

# Initialize FPGA sum and timing
fpga_sum = 0
fpga_start_time = time.time()

# Process each chunk
for i in range(num_chunks):
    print(i)
    start_idx = i * chunk_size
    end_idx = start_idx + chunk_size
    chunk = array_t[start_idx:end_idx]

    # Allocate buffers for input and output
    with allocate(shape=(chunk_size,), dtype=np.int32) as in_buffer, \
         allocate(shape=(1,), dtype=np.int32) as out_buffer:
        
        # Copy chunk data to input buffer
        np.copyto(in_buffer, chunk)
        
        # Transfer data to FPGA
        dma.sendchannel.transfer(in_buffer)
        dma.recvchannel.transfer(out_buffer)
        
        # Wait for DMA transfer to complete
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        
        # Retrieve partial sum from FPGA
        partial_sum = out_buffer[0]
        fpga_sum += partial_sum

        # Optional: Print progress
        if (i+1) % 10 == 0 or (i+1) == num_chunks:
            print(f"Processed chunk {i+1}/{num_chunks}")

# Handle any remaining elements that don't fit into a full chunk
if remaining_elements > 0:
    start_idx = num_chunks * chunk_size
    chunk = array_t[start_idx:start_idx + remaining_elements]
    
    # Allocate buffers for the remaining elements
    with allocate(shape=(remaining_elements,), dtype=np.int32) as in_buffer, \
         allocate(shape=(1,), dtype=np.int32) as out_buffer:
        
        np.copyto(in_buffer[:remaining_elements], chunk)
        
        # Transfer data to FPGA
        dma.sendchannel.transfer(in_buffer[:remaining_elements])
        dma.recvchannel.transfer(out_buffer)
        
        # Wait for DMA transfer to complete
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        
        # Retrieve partial sum from FPGA
        partial_sum = out_buffer[0]
        fpga_sum += partial_sum
        
    print(f"Processed remaining {remaining_elements} elements.")

fpga_end_time = time.time()
fpga_exe_time = fpga_end_time - fpga_start_time

# Display results
print("\nResults:")
print(f"CPU Sum: {cpu_sum}")
print(f"FPGA Sum: {fpga_sum}")
print(f"CPU Execution Time: {cpu_exe_time:.4f} seconds")
print(f"FPGA Execution Time: {fpga_exe_time:.4f} seconds")
if fpga_exe_time > 0:
    speedup = cpu_exe_time / fpga_exe_time
    print(f"FPGA is {speedup:.2f} times faster than CPU.")
else:
    print("FPGA execution time is too fast to calculate speedup.")


In [14]:
print(0.06855249404907227/0.010870933532714844)

6.3060356171594


In [9]:
import pynq
from pynq import Overlay
from pynq.lib.dma import DMA
import numpy as np
from pynq import allocate
import time

# Load the FPGA overlay
overlay = Overlay("sumAll.bit")
dma = overlay.axi_dma_0

# Generate random numbers between 1 and 10, size=100
total_elements = 100
random_numbers = np.random.randint(1, 10, size=total_elements, dtype=np.int32)


In [10]:

# Compute the sum on the CPU for verification
print(f"Total number of elements: {len(random_numbers)}")
cpu_start_time = time.time()
cpu_sum = np.sum(random_numbers)
cpu_end_time = time.time()
cpu_exe_time = cpu_end_time - cpu_start_time
print("CPU sum result:", cpu_sum)
print(f"CPU execution time: {cpu_exe_time:.6f} seconds\n")

# Prepare data for FPGA processing
array_t = np.array(random_numbers, dtype=np.int32)
chunk_size = 10  # Number of elements per chunk
num_chunks = total_elements // chunk_size
remaining_elements = total_elements % chunk_size

# Initialize FPGA sum and timing
fpga_sum = 0
fpga_start_time = time.time()



Total number of elements: 100
CPU sum result: 520
CPU execution time: 0.001227 seconds



In [12]:
chunk = array_t[0:chunk_size]

# Allocate buffers for input and output
with allocate(shape=(chunk_size,), dtype=np.int32) as in_buffer, \
     allocate(shape=(1,), dtype=np.int32) as out_buffer:

    # Copy chunk data to input buffer
    np.copyto(in_buffer, chunk)

    # Transfer data to FPGA
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)

    # Wait for DMA transfer to complete
    dma.sendchannel.wait()
    dma.recvchannel.wait()

    # Retrieve partial sum from FPGA
    partial_sum = out_buffer[0]
    fpga_sum += partial_sum

    # Optional: Print progress
    print(f"Processed chunk {1}/{num_chunks}: Partial Sum = {partial_sum}")



KeyboardInterrupt: 

In [ ]:
import pynq
from pynq import Overlay
from pynq.lib.dma import DMA
import numpy as np
from pynq import allocate
import time

# Load the FPGA overlay
overlay = Overlay("sumAll.bit")
dma = overlay.axi_dma_0

# Generate random numbers between 1 and 10, size=100
total_elements = 100
random_numbers = np.random.randint(1, 10, size=total_elements, dtype=np.int32)

# Compute the sum on the CPU for verification
print(f"Total number of elements: {len(random_numbers)}")
cpu_start_time = time.time()
cpu_sum = np.sum(random_numbers)
cpu_end_time = time.time()
cpu_exe_time = cpu_end_time - cpu_start_time
print("CPU sum result:", cpu_sum)
print(f"CPU execution time: {cpu_exe_time:.6f} seconds\n")

# Prepare data for FPGA processing
array_t = np.array(random_numbers, dtype=np.int32)
chunk_size = 10  # Number of elements per chunk
num_chunks = total_elements // chunk_size
remaining_elements = total_elements % chunk_size

# Initialize FPGA sum and timing
fpga_sum = 0
fpga_start_time = time.time()

# Process each chunk in a loop
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = start_idx + chunk_size
    chunk = array_t[start_idx:end_idx]

    # Allocate buffers for input and output within the loop
    with allocate(shape=(chunk_size,), dtype=np.int32) as in_buffer, \
         allocate(shape=(1,), dtype=np.int32) as out_buffer:
        
        # Copy chunk data to input buffer
        np.copyto(in_buffer, chunk)
        
        # Transfer data to FPGA
        dma.sendchannel.transfer(in_buffer)
        dma.recvchannel.transfer(out_buffer)
        
        # Wait for DMA transfer to complete
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        
        # Retrieve partial sum from FPGA
        partial_sum = out_buffer[0]
        fpga_sum += partial_sum

        # Optional: Print progress
        print(f"Processed chunk {i+1}/{num_chunks}: Partial Sum = {partial_sum}")

# Handle any remaining elements that don't fit into a full chunk
if remaining_elements > 0:
    start_idx = num_chunks * chunk_size
    chunk = array_t[start_idx:start_idx + remaining_elements]
    
    # Allocate buffers for the remaining elements
    with allocate(shape=(remaining_elements,), dtype=np.int32) as in_buffer, \
         allocate(shape=(1,), dtype=np.int32) as out_buffer:
        
        # Copy remaining chunk data to input buffer
        np.copyto(in_buffer[:remaining_elements], chunk)
        
        # Transfer data to FPGA
        dma.sendchannel.transfer(in_buffer[:remaining_elements])
        dma.recvchannel.transfer(out_buffer)
        
        # Wait for DMA transfer to complete
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        
        # Retrieve partial sum from FPGA
        partial_sum = out_buffer[0]
        fpga_sum += partial_sum
    
    print(f"Processed remaining {remaining_elements} elements: Partial Sum = {partial_sum}")

fpga_end_time = time.time()
fpga_exe_time = fpga_end_time - fpga_start_time

# Display results
print("\nResults:")
print(f"CPU Sum: {cpu_sum}")
print(f"FPGA Sum: {fpga_sum}")
print(f"CPU Execution Time: {cpu_exe_time:.6f} seconds")
print(f"FPGA Execution Time: {fpga_exe_time:.6f} seconds")

if fpga_exe_time > 0:
    speedup = cpu_exe_time / fpga_exe_time
    print(f"FPGA is {speedup:.2f} times faster than CPU.")
else:
    print("FPGA execution time is too fast to calculate speedup.")
